In [ ]:
%pip install unidecode
%pip install pybaseball

In [4]:
import warnings
warnings.filterwarnings('ignore')
from pybaseball import statcast
from pybaseball import playerid_reverse_lookup
from unidecode import unidecode 
import os
import json
import time
import pickle

## Scraping Data
Finding a good data source for training data is very hard.  You need massive amounts of data. One of the approaches is to use derived text to train a specific model.  You can turn statistical data into sturctured data by encoding the data with a complex transformation.  Below I take twenty years of baseball data and encode it into a structured data.  This way we use llm to generate structured data a program can understand. 

We first download and cache all the the mlb statcast events from the start of the season to the end of the season from 2003 to 2023 (this will take a long time)

In [ ]:

years = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2018, 2019, 2020, 2021, 2022, 2023]
for year in years:
    events = statcast(start_dt=f"{year}-03-01", end_dt=f"{year}-11-15")
    with open(f"../data/seasons/season-{year}.pickle", "wb") as scores:
        pickle.dump(events, scores)
    time.sleep(5)

Here comes the fun part!  We must examine this data to see what it contains and what we can use to encode the information we want to know.  We want to know the outcome of each pitch to the batter and use that to build a response that contains the at-bat of each player and the subsequent event or the outcome of the at-bat.  

But first we use the cache to create a dictionary of the player UUIDs to their actual names




In [15]:
players = {}

In [ ]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
for file in files:
    print(file)
    pickle_in = open(f"{path}/{file}","rb")
    every_pitch = pickle.load(pickle_in)
    every_pitch = every_pitch.iloc[::-1]
    for index, row in every_pitch.iterrows(): 
        batter = str(row.batter)
        pitcher = str(row.pitcher)
        if pitcher not in players:
            name_data = playerid_reverse_lookup([row.pitcher], key_type='mlbam')
            if len(name_data['name_first']) > 0 and len(name_data['name_last']) > 0:
                players[pitcher] = f"{unidecode(name_data['name_first'][0])} {unidecode(name_data['name_last'][0])}"
            else:
                players[pitcher] = pitcher
        if batter not in players:
            name_data = playerid_reverse_lookup([row.batter], key_type='mlbam')
            if len(name_data['name_first']) > 0 and len(name_data['name_last']) > 0:
                players[batter] = f"{unidecode(name_data['name_first'][0])} {unidecode(name_data['name_last'][0])}"
            else:
                players[batter] = batter
       
with open(f'../data/players.json', 'w', encoding='utf-8') as f:
    json.dump(players, f, ensure_ascii=True, indent=4, allow_nan=True)
    f.close()


In [5]:
with open(f'../data/players.json', 'r', encoding='utf-8') as f:
    players = json.load(f)

In [6]:
from decimal import Decimal
from pandas._libs.missing import NAType

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if obj is None:
            return ""
        if isinstance(obj, Decimal):
            return str(obj)
        if isinstance(obj, NAType):
            return ""
        # 👇️ otherwise use the default behavior
        return json.JSONEncoder.default(self, obj)

In [7]:
import random

In [15]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
baseball = []
input_data = []
results_data = []
lines = []
json_data = {}
json_data['train'] = {}
json_data['test'] = {}
json_data['valuation'] = {}
json_data['train']['text'] = []
json_data['test']['text'] = []
json_data['valuation']['text'] = []
test = []
season_count = 0
for file in files:
    print(file)
    if "2003" not in file:
        pickle_in = open(f"{path}/{file}","rb")
        every_pitch = pickle.load(pickle_in)
        every_pitch = every_pitch.iloc[::-1]
        season = []
        at_bat = []
        pre_batter = ""
        for index, row in every_pitch.iterrows():
            current_batter = str(row.batter)
            if current_batter in players and str(row.pitcher) in players:
                if current_batter not in pre_batter:
                    pre_batter = current_batter
                    at_bat = []
                    pitch_type = []
                    release_speed = []
                    pitch_names = []
                    description = []


                at_bat.append(row.description)
                release_speed.append(row.release_speed)
                pitch_names.append(row.pitch_name)

                if isinstance(row.events, str):
                    on_1b = row.on_1b
                    if str(row.on_1b) in players:
                        on_1b = {
                            'id':row.on_1b,
                            "name":players[str(row.on_1b)]
                        }

                    on_2b = row.on_2b
                    if str(row.on_2b) in players:
                        on_2b = {
                            'id':row.on_2b,
                            "name":players[str(row.on_2b)]
                        }

                    on_3b = row.on_3b
                    if str(row.on_3b) in players:
                        on_3b = {
                            'id':row.on_3b,
                            "name":players[str(row.on_3b)]
                        }


                    input = {
                                'pitcher':{
                                    'id':row.pitcher,
                                    'name':players[str(row.pitcher)]
                                },
                                'batter':{
                                    'id':row.batter,
                                    'name':players[str(row.batter)]
                                },
                                "at_bat_number":row.at_bat_number,
                                "p_throws":row.p_throws,
                                "stand":row.stand,
                                "inning_topbot":row.inning_topbot,
                                "inning":row.inning,
                                "outs_when_up":row.outs_when_up,
                                "on_1b":on_1b,
                                "on_2b":on_2b,
                                "on_3b":on_3b,
                                "home_score":row.home_score,
                                "away_score":row.away_score,
                                "pitch_number":row.pitch_number,
                                'if_fielding_alignment':row.if_fielding_alignment,
                                'of_fielding_alignment':row.of_fielding_alignment
                            }

                    result = {
                                "event":row.events,
                                "event_description":row.description,
                                "pitch_name":row.pitch_name,
                                "description":unidecode(row.des),
                                "runs":row.post_bat_score-row.bat_score,
                                "at_bat":at_bat,
                                "release_speeds":release_speed,
                                "pitch_names":pitch_names
                            }
                    input_data.append({
                        "input":input,
                    })
                    results_data.append({
                        "result":result
                    }
                    )
                    baseball.append(
                        {
                            "Instruct":input,
                            "Output":result
                        }
                    )
                    

                    if random.randint(0, 341543512) < 341543512/100000:
                        json_data['test']['text'].append(f'''Instruct: {json.dumps(input, ensure_ascii=True, allow_nan=True, cls=MyEncoder)}. \n Output: {json.dumps(result, ensure_ascii=True, allow_nan=True, cls=MyEncoder)}\n''')
                    elif random.randint(0, 341543512) < 341543512/10000:
                        json_data['valuation']['text'].append(f'''Instruct: {json.dumps(input, ensure_ascii=True, allow_nan=True, cls=MyEncoder)}. \n''')
                    else:
                        json_data['train']['text'].append(f'''Instruct: {json.dumps(input, ensure_ascii=True, allow_nan=True, cls=MyEncoder)}. \n Output: {json.dumps(result, ensure_ascii=True, allow_nan=True, cls=MyEncoder)}\n''')
                            
        
        season_count+=1

        with open(f'2011_2023_train_test_valuation.json', 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=True, indent=4, allow_nan=True, cls=MyEncoder)
            f.close()
        
        with open(f'2011_2023_Instruct_Output.json', 'w', encoding='utf-8') as f:
            json.dump(baseball, f, ensure_ascii=True, indent=4, allow_nan=True, cls=MyEncoder)
            f.close()

# with open(f'twenty_years_of_baseball_question_answer.txt', 'w', encoding='utf-8') as f:
#     f.writelines(lines)
#     f.close()

season-2011.pickle
season-2012.pickle
season-2014.pickle
season-2015.pickle
season-2016.pickle
season-2018.pickle
season-2019.pickle
season-2020.pickle
season-2021.pickle
season-2022.pickle
season-2023.pickle


In [47]:
path = "../data/seasons/"
files = os.listdir(path)
files.sort()
baseball = {}
baseball['data'] = {}
baseball['data']['train'] = []
input_data = []
results_data = []
lines = []
json_data = {}
json_data['data'] = {}
json_data['data']['train'] = []
json_data['data']['test'] = []
test = []
season_count = 0
for file in files:
    print(file)
    if "2003" not in file:
        pickle_in = open(f"{path}/{file}","rb")
        every_pitch = pickle.load(pickle_in)
        every_pitch = every_pitch.iloc[::-1]
        season = []
        at_bat = []
        pre_batter = ""
        for index, row in every_pitch.iterrows():
            current_batter = str(row.batter)
            if current_batter in players and str(row.pitcher) in players:
                if current_batter not in pre_batter:
                    pre_batter = current_batter
                    # at_bat = []
                    # pitch_type = []
                    # release_speed = []
                    # pitch_name = []
                    # description = []

                # at_bat.append(row.description)
                # pitch_type.append(row.pitch_type)
                # release_speed.append(row.release_speed)
                # pitch_name.append(row.pitch_name)
                
                if isinstance(row.events, str):
                    # on_1b = row.on_1b
                    # if str(row.on_1b) in players:
                    #     on_1b = {
                    #         'id':row.on_1b,
                    #         "name":players[str(row.on_1b)]
                    #     }

                    # on_2b = row.on_2b
                    # if str(row.on_2b) in players:
                    #     on_2b = {
                    #         'id':row.on_2b,
                    #         "name":players[str(row.on_2b)]
                    #     }

                    # on_3b = row.on_3b
                    # if str(row.on_3b) in players:
                    #     on_3b = {
                    #         'id':row.on_3b,
                    #         "name":players[str(row.on_3b)]
                    #     }


                    # input = {
                    #             'pitcher':{
                    #                 'id':row.pitcher,
                    #                 'name':players[str(row.pitcher)]
                    #             },
                    #             'batter':{
                    #                 'id':row.batter,
                    #                 'name':players[str(row.batter)]
                    #             },
                    #             # "p_throws":row.p_throws,
                    #             # "stand":row.stand,
                    #             # "inning_topbot":row.inning_topbot,
                    #             # "inning":row.inning,
                    #             # "outs_when_up":row.outs_when_up,
                    #             # "on_1b":on_1b,
                    #             # "on_2b":on_2b,
                    #             # "on_3b":on_3b,
                    #             # "home_score":row.home_score,
                    #             # "away_score":row.away_score
                    #         }

                    # result = {
                    #             "event":row.events,
                    #             # "type":row.type,
                    #             # "zone":row.zone,
                    #             "des":unidecode(row.des),
                    #             # "at_bat_number":row.at_bat_number,
                    #             # "pitch_number":row.pitch_number,
                    #             # "pitch_name":row.pitch_name,
                    #             # "hit_location":row.hit_location,
                    #             # "launch_speed":row.launch_speed,
                    #             # "launch_speed_angle":row.launch_speed_angle,
                    #             # "runs_scored":row.post_bat_score - row.bat_score,
                    #             # "zone":row.zone,
                    #             # "at_bat":at_bat,
                    #             # "pitch_type":pitch_type,
                    #             # "release_speed":release_speed,
                    #             # "pitch_name":pitch_name,
                    #         }

                    json_data['data']['train'].append(f'''Instruct: What is the outcome of {players[str(row.pitcher)]} pitching to {players[str(row.batter)]}. \n Output: {row.events}\n''') 
        
        season_count+=1

        with open(f'2011_2023_row.events.json', 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=True, indent=4, allow_nan=True, cls=MyEncoder)
            f.close()

season-2011.pickle
season-2012.pickle
season-2014.pickle
season-2015.pickle
season-2016.pickle
season-2018.pickle
season-2019.pickle
season-2020.pickle
season-2021.pickle
season-2022.pickle
season-2023.pickle


In [22]:
import random

In [44]:
with open(f'twenty_years_of_baseball_structed_train_lines_text.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
    train = []
    eval = []
    for i in json_data['data']['train']:
        train.append(i)

    data = {
        "data":{
            "text":train,
        }
    }
    with open(f'train.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=True, indent=4, allow_nan=True, cls=MyEncoder)
        f.close()
                        

In [43]:
import csv
import json
def convert_csv_to_json(csv_file_path):
    # Read CSV file
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)
        rows = list(reader)
    # Convert CSV data to JSON
    json_data = json.dumps(rows, indent=4)

    # Save JSON data to a file (optional)
    with open('../data/2018_Central_Park_Squirrel_Census_-_Stories_20240204.json', 'w') as json_file:
        json_file.write(json_data)

    return json_data

# Specify the path to your CSV file
csv_file_path = '../data/2018_Central_Park_Squirrel_Census_-_Stories_20240204.csv'

# Convert CSV to JSON
json_data = convert_csv_to_json(csv_file_path)

print("Conversion completed. JSON data:")
# print(json_data)
json_data = json.loads(json_data)
squire_stories = {}
squire_stories['data'] = {}
squire_stories['data']['train'] = []
for i in json_data:
    squire_stories['data']['text'].append(f"Question: What is an observation about a squire in central park? \n Output: {unidecode(i['Note Squirrel & Park Stories'])}\n")

with open(f'squires.json', 'w', encoding='utf-8') as f:
    json.dump(squire_stories, f, ensure_ascii=True, indent=4, allow_nan=True, cls=MyEncoder)
    f.close()

Conversion completed. JSON data:


KeyError: 'text'

In [ ]:
""